In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
import spacy
import pandas as pd
import re
import numpy as np

In [ ]:
import pandas as pd
data = pd.read_csv("/content/data_en.csv")
data.head()

In [ ]:
data.text = data.text.apply(lambda x: x.lower())
data.head()

In [ ]:
data.shape

In [ ]:
data.sentiment.value_counts()

In [ ]:
data.isnull().sum()

In [ ]:
data.duplicated().sum()

In [ ]:
#data.sentiment.replace({"Positive":1,"Negative":0},inplace=True)
#data.head()

In [ ]:
nlp = spacy.load('en_core_web_lg')

def text_preprocessing(text):
  lower = text.lower()
  doc = nlp(lower)
  tokens = []
  for token in doc:
    if token.pos_ not in ["SPACE","PUNCT","X"]:
      tokens.append(token.lemma_)
  return " ".join(tokens)



In [ ]:
data.head()

In [ ]:
data.text = data.text.apply(text_preprocessing)
data.head()

In [ ]:
data.to_csv("/content/drive/MyDrive/data_en_preprocessed.csv",index=False)

In [ ]:
data = pd.read_csv("/content/data_en_preprocessed.csv")
data.head()

In [ ]:
X = data.text
y = data.sentiment

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2, stratify=y, random_state=42)

In [ ]:
train_data.text[0]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
X_train[0]

In [ ]:
X_train.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
# Création du pipeline
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),  # Étape de vectorisation du texte
    ('classifier', LogisticRegression())  # Modèle de classification
])

# Entraînement du pipeline
pipeline.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import accuracy_score
# Prédiction
y_pred = pipeline.predict(X_test)

# Évaluation
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
accuracy

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

In [ ]:
from xgboost import XGBClassifier
pip_line = Pipeline([
    ('vectorizer', TfidfVectorizer()),  # Étape de vectorisation du texte
    ('classifier', XGBClassifier(n_estimators=200))  # Modèle de classification
])

# Entraînement du pipeline
pip_line.fit(X_train, y_train)

In [ ]:
y_pred = pip_line.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
pipline = Pipeline([
    ('vectorizer', TfidfVectorizer()),  # Étape de vectorisation du texte
    ('classifier', RandomForestClassifier())  # Modèle de classification
])

# Entraînement du pipeline
pipline.fit(X_train, y_train)

In [ ]:
y_pred = pipline.predict(X_test)
print(classification_report(y_test,y_pred))

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:

# 1. Tokenize the text data
tokenizer = Tokenizer(num_words=100000)  # Limit vocabulary size
tokenizer.fit_on_texts(data["text"])
sequences = tokenizer.texts_to_sequences(data["text"])

# 2. Pad sequences to ensure uniform length
X = pad_sequences(sequences, maxlen=100)  # Adjust maxlen as needed

# 3. Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
X

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

# Création du modèle LSTM avec Dropout
lstm = Sequential([
    Embedding(input_dim=10000, output_dim=16, input_length=100),  # Couche d'embedding
    Bidirectional(LSTM(64, return_sequences=True)),  # LSTM bidirectionnelle avec retour des séquences
    Dropout(0.5),  # Dropout de 50% après la première couche LSTM
    Bidirectional(LSTM(64)),  # Deuxième couche LSTM bidirectionnelle
    Dropout(0.5),  # Dropout de 50% après la deuxième couche LSTM
    Dense(64, activation="relu"),  # Couche dense
    Dropout(0.5),  # Dropout de 50% avant la couche de sortie
    Dense(1, activation='sigmoid')  # Sortie avec activation sigmoid pour classification binaire
])

# Compilation du modèle
lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
lstm.fit(X_train,y_train,epochs=10, batch_size=100 ,validation_data=(X_test,y_test))

In [ ]:
lstm.evaluate(X_test,y_test)

In [ ]:
from sklearn.metrics import classification_report
y_pred = lstm.predict(X_test)
y_pred = np.where(y_pred > 0.5, 1, 0)
print(classification_report(y_test,y_pred))

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Exemple de phrase à tester
phrase = "I'm glad to see you!"

# Tokenisation de la phrase
sequence = tokenizer.texts_to_sequences([phrase])

# Padding de la séquence pour qu'elle ait une longueur de 100 (comme défini dans le modèle)
padded_sequence = pad_sequences(sequence, maxlen=100)

# Faire une prédiction avec le modèle LSTM
prediction = lstm.predict(padded_sequence)

# Afficher le résultat
print(f"Prédiction : {prediction}")


In [ ]:
gru = Sequential([
    Embedding(input_dim=10000, output_dim=16, input_length=100),
    Bidirectional(tf.keras.layers.GRU(64, return_sequences=True)),
    Bidirectional(tf.keras.layers.GRU(64)),
    Dense(64, activation="relu"),  # Couche dense
    Dense(1, activation='sigmoid')
])
gru.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
gru.summary()

In [ ]:
gru.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [ ]:
gru.evaluate(X_test,y_test)

In [ ]:
y_pred = gru.predict(X_test)
y_pred = np.where(y_pred > 0.5, 1, 0)
print(classification_report(y_test,y_pred))

In [ ]:
df = data.copy()
df.head()

In [ ]:
df.info()

In [ ]:
df["label"] = "__label__" + df["sentiment"].astype(str)
df.head()

In [ ]:
df["data"] = df["label"] + " " + df["text"]
df.head()

In [ ]:
df.drop(["sentiment","label","text"],axis=1,inplace=True)
df.head()

In [ ]:
train_fasttext_en, test_fasttext_en = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
train_fasttext_en.to_csv("fasttext_en_train.train",index=False,header=False)
test_fasttext_en.to_csv("fasttext_en_test.test",index=False,header=False)

In [ ]:
!pip install fasttext

In [ ]:
import fasttext

model = fasttext.train_supervised(input="/content/fasttext_en_train.train")

In [ ]:
model.test("/content/fasttext_en_test.test")

In [ ]:
model.predict("I love you")

In [ ]:
!pip install transformers

In [ ]:
!pip install torch

In [ ]:
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, texts, labels, tokenizer, max_len=512):
    self.texts = texts
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    text = str(self.texts[idx])
    label = torch.tensor(self.labels[idx])

    encoding = self.tokenizer.encode_plus(text, truncation=True,
                                          padding='max_length',
                                          max_length=self.max_len
                                          )
    return {
        'input_ids': encoding['input_ids'],
        'attention_mask': encoding['attention_mask'],
        'labels': label
    }


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2).to(device)

In [ ]:
model

In [ ]:
X = data["text"].tolist()
y = data["sentiment"].tolist()

dataset = CustomDataset(X, y, tokenizer)

In [ ]:
dataset[0].keys()

In [ ]:
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
def compute_metrics(exemple):
    labels = exemple.label_ids
    preds = exemple.predictions.argmax(-1)

    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')

    return {'accuracy': accuracy, 'f1': f1}

In [ ]:
from transformers import Trainer, TrainingArguments
batch_size = 16
model_name = "distilbert-base-uncased"

args = TrainingArguments(
    output_dir="output",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=2e-5,
    num_train_epochs=1,
    evaluation_strategy="epoch"
)


In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [ ]:
import os
os.environ['WANDB_MODE'] = 'disabled'


In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model(model_name)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')